In [1]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
import time
from PIL import Image
from pytesseract import pytesseract
from io import BytesIO
import base64
import pytesseract

In [2]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'  # your path may be different

In [3]:
global data

In [4]:
#get text from image
def getFeatureTextFromImg(imgUrl):
    url = imgUrl
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    img = Image.open(BytesIO(r.content))
    text = pytesseract.image_to_string(img)
    return text

In [94]:
txt=getFeatureTextFromImg('https://ksa.carswitch.com/assets/png/43.png')
print(txt)

In [91]:
def getDetailsData(detailsUrl):
    #print(detailsUrl)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    detialRequest = requests.get(detailsUrl,headers=headers)
    detailContent = detialRequest.content
    detailSoup = BeautifulSoup(detailContent,features="html.parser")
    details_container=detailSoup.find('aside',{'class':'left-side'})
    #location=details_container.find('div',{'class','location'})
    return details_container


In [132]:
def getCarsData(page_from,page_to):
    global df
    data=[]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    for pageNo in range(page_from,page_to):
        print(pageNo)
        url='https://ksa.carswitch.com/en/saudi/used-cars/search?page={}'.format(pageNo)
        r = requests.get(url,headers=headers)
        content = r.content
        soup = BeautifulSoup(content,features="html.parser")
        
        #soup = BeautifulSoup(data,'html.parser')
        #print(soup)
        #get div with id main-listing-div
        soup = soup.find('section',{'id':'main-listing-div'})
        #print(soup)
        features={}
        items=soup.findAll('div',{'class':'show-price-badge'})
        #print(items)
        for index,item in enumerate(items):
            features={}
            title=item.find('div',{'class':'title'}).find('a').text
            price=item.find('div',{'class':'title'}).find('span').text
            model_year=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[0]
            odo=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[1]
            #get more data from details page
            detailsUrl='https://ksa.carswitch.com'+item.find('figure',{'class','image-holder'}).find('a',href=True)['href']
            print(detailsUrl)
            details_html=getDetailsData(detailsUrl)
            location=details_html.find('div',{'class','location'}).find('span').text
            features_html=details_html.findAll('div',{'class','car-overview__features'})
            #print(features_html)
            #print(features_html.length)
            features=[]
            #print(features_html[:-1])
            for category in features_html[:-1]:
                for index,feature in enumerate(category.findAll('div',class_='item')):
                    if len(feature["class"]) != 1:#if the div has more than one class
                        continue
                    else:
                        img=feature.find('img')
                        if img.has_attr('data-src'):
                            imgUrl="https://ksa.carswitch.com"+img['data-src']
                        else:
                            imgUrl="https://ksa.carswitch.com"+img['src']
                        imgText=getFeatureTextFromImg(imgUrl)
                        if imgText == '' and feature.has_attr('data-searchable'):
                            features.append(feature['data-searchable'] )
                        else:
                            features.append(imgText.replace("\n"," ").strip().replace("\x0c", "") )
                        #print(imgText)
            try:
                print(index,title,'   |',price,'  |',model_year,'  |',odo,' |',location," |",features)
                
            except Exception as ex :
                print(ex)
                continue

In [134]:
time.time()

1658011824.1585748

In [135]:
t0 = time.time()
getCarsData(1,2)
t1 = time.time()
print('\n\nTime comsumed: ', t1-t0)

1
https://ksa.carswitch.com/en/dammam/used-car/chevrolet/camaro/2014/308388-312219
5 Chevrolet Camaro V8 6.2L SS     | 
SAR 55,000    | 2014    | 79,000 KM  | 
Industrial Area Al Sinayia, Dammam   | ['fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', 'Steering wheel control', 'oP  Auto dimming mirror', '>  Bluetooth', '{0} premium sound system', 'ic  Leather Seats', 'Remote start', '(sie)  LED headlights']
https://ksa.carswitch.com/en/riyadh/used-car/toyota/camry/2020/281146-284959
5 Toyota Camry I4 2.5L    | 
SAR 65,000    | 2020    | 46,000 KM  | 
As Suwaidi, Riyadh   | ['>  Bluetooth', '{0} premium sound system', 'A  So.  Collision warning system', '(sie)  LED headlights']
https://ksa.carswitch.com/en/riyadh/used-car/mercedes/c200/2015/320111-323943
5 Mercedes C200 I4 2.0L AMG    | 
SAR 81,500    | 2015    | 109,000 KM  | 
Al Rawdhah, Riyadh   | ['fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', '{Or Automated Parking', 'A

5 Mercedes S550 V8 4.7L     | 
SAR 183,000    | 2015    | 65,000 Miles  | 
Al Marwah, Jeddah   | ['fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', '8  Birds eye view camera', 'Gy C__)  1  Rear mirror camera', '{Or Automated Parking', '(S)  Adaptive cruise control', 'A-©  Digital display', 'Steering wheel control', 'Paddle shifters', 'oP  Auto dimming mirror', '>  Bluetooth', '{0} premium sound system', 'ic  Leather Seats', 'Push Start', 'Remote start', 'vi.  Ambient lighting', 'Keyless entry', '\\s  Power seats', 'IS  Memory seats', 'i  ey  Power folding mirrors', 'Soft close door', '&  Hands free liftgate', '18  Heated-cooled seats', 'SN ge Blind spot monitor', 'g  Lane keep assist', 'A  So.  Collision warning system', '(sie)  LED headlights', 'OF  Fog lights', 'Sy  Brake assist']
https://ksa.carswitch.com/en/riyadh/used-car/porsche/cayenne/2016/321096-324928
5 Porsche Cayenne V6 3.6L GTS    | 
SAR 162,500    | 2016    | 196,000 KM  | 
Al Wadi, Riyadh  

In [ ]:
timeit.timeit('foobar(x,y)', number=1000, globals = globals())

In [ ]:
https://ksa.carswitch.com/en/saudi/used-cars/search

In [ ]:
pro-item show-price-badge cash  ksa badges-visible

In [76]:
'2017 · 210,000 km'.split(' · ')[0]

'2017'